<a href="https://colab.research.google.com/github/PrakritiShetty/DS303_Project_Paper_Implementation/blob/main/Paper_Implementation_DS303.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Downloading and Preprocessing

In [ ]:
import os
import numpy as np
import random
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import albumentations as A
import sys

In [ ]:
from PIL import Image

IMAGE_FORMAT = ".png"
# generating LR images from HR by bicubic downsampling. Average is simple avg, bicubic is weighted avg and subsampling is subsitution.
DOWNSAMPLE_MODE = Image.BICUBIC
COLOR_CHANNELS = 3

HR_IMG_SIZE = (648, 648) 
UPSCALING_FACTOR = 4
LR_IMG_SIZE = (HR_IMG_SIZE[0] // UPSCALING_FACTOR, HR_IMG_SIZE[1] // UPSCALING_FACTOR) # used // for integer division

In [ ]:
class DIV2K_Dataset(keras.utils.Sequence):
   
   # keras.utils.sequence is a data generator - used in situations like when we need advanced control over sample generation or when simple data does not fit into memory and must be loaded dynamically
    
    def __init__(self, hr_image_folder: str, batch_size: int, set_type: str):
        self.batch_size = batch_size
        self.hr_image_folder = hr_image_folder
        self.images = np.sort([
            x for x in os.listdir(hr_image_folder) if x.endswith(IMAGE_FORMAT)
        ])

        if set_type == "train":
          self.images = self.images[:-200] # 700 images for training
        elif set_type == "val":
          self.images = self.images[-200:-100] # 100 images for validation
        else:
          self.images = self.images[-100:] # 100 images for testing

        # data augmentation
        # done on HR images only, then LR will be made from that.
        # for training and validation sets, data augmentation includes scaling and rotation
        if set_type in ["train", "val"]:
            self.transform = A.Compose(
                [
                    A.RandomCrop(width=HR_IMG_SIZE[0], height=HR_IMG_SIZE[1], p=1.0),
                    A.RandomRotate90(),
                    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.90,rotate_limit=45, p=.75),
                    A.OneOf([A.augmentations.geometric.resize.RandomScale (scale_limit=0.6, interpolation=1, always_apply=False, p=0.5)],[A.augmentations.geometric.resize.RandomScale (scale_limit=0.7, interpolation=1, always_apply=False, p=0.5)][A.augmentations.geometric.resize.RandomScale (scale_limit=0.8, interpolation=1, always_apply=False, p=0.5)][A.augmentations.geometric.resize.RandomScale (scale_limit=0.9, interpolation=1, always_apply=False, p=0.5)]),
                    A.OneOf([A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5)])
                    
                ]
            )
        else: 
            self.transform = A.Compose(
                [
                    A.RandomCrop(width=HR_IMG_SIZE[0], height=HR_IMG_SIZE[1], p=1.0),
                ]
            )

        self.to_float = A.ToFloat(max_value=255)

    def __len__(self):
        return len(self.images) // self.batch_size

    def on_epoch_end(self):
        random.shuffle(self.images)

    def __getitem__(self, idx):
        # batch of samples
        i = idx * self.batch_size
        batch_images = self.images[i : i + self.batch_size]
        batch_hr_images = np.zeros((self.batch_size,) + HR_IMG_SIZE + (COLOR_CHANNELS,))
        batch_lr_images = np.zeros((self.batch_size,) + LR_IMG_SIZE + (COLOR_CHANNELS,))


        for i, image in enumerate(batch_images):
           
            hr_image = np.array(Image.open(os.path.join(self.hr_image_folder, image)))  

                 
            hr_image_transform = self.transform(image=hr_image)["image"]
            hr_image_transform_1 = Image.fromarray(hr_image_transform)
            lr_image_transform = hr_image_transform_1.resize(
                LR_IMG_SIZE, resample=DOWNSAMPLE_MODE
            )
            lr_image_transform_1 = Image.fromarray(lr_image_transform)

            batch_hr_images[i] = self.to_float(image=hr_image_transform)["image"]
            batch_lr_images[i] = self.to_float(image=lr_image_transform)["image"]

        return (batch_lr_images, batch_hr_images)

    

Model Building

In [ ]:
from keras import Sequential, initializers
from keras.layers import Conv2D, Conv2DTranspose, InputLayer, PReLU, Activation

# from utils.constants import LR_IMG_SIZE, UPSCALING_FACTOR, COLOR_CHANNELS

In [ ]:
def create_model( d: int, s: int, m: int, input_size: tuple = LR_IMG_SIZE, upscaling_factor: int = UPSCALING_FACTOR, color_channels: int = COLOR_CHANNELS):
    model = Sequential()
    model.add( InputLayer( input_shape=(input_size[0], input_size[1], color_channels)))

    # feature extraction
    model.add(
        Conv2D(
            kernel_size = 5,
            filters = d,
            padding="valid",
            kernel_initializer=initializers.HeNormal(),
        )
    )

    # activation func after every conv layer
    model.add( PReLU( alpha_initializer="zeros", shared_axes=[1, 2]))

    # shrinking
    model.add(
        Conv2D(
            kernel_size = 1,
            filters = s,
            padding="valid",
            kernel_initializer=initializers.HeNormal(),
        )
    )

    model.add( PReLU( alpha_initializer="zeros", shared_axes=[1, 2]))

    # non linear mapping
    for _ in range(m):
        model.add(
            Conv2D(
                kernel_size = 3,
                filters = s,
                padding="valid",
                kernel_initializer=initializers.HeNormal(),
            )
        )
    
    model.add(PReLU(alpha_initializer="zeros", shared_axes=[1, 2]))

    # expanding
    model.add(
        Conv2D(
            kernel_size=1, 
            filters=d, 
            padding="valid"
          )
      )
    
    model.add(PReLU(alpha_initializer="zeros", shared_axes=[1, 2]))

    # deconvolution
    model.add(
        Conv2DTranspose(
            kernel_size=9,
            filters=color_channels,
            strides=upscaling_factor,
            padding="valid",
            kernel_initializer=initializers.RandomNormal(mean=0, stddev=0.001),
        )
    )

    return model
